In [1]:
import pandas as pd
import clickhouse_connect
import os

In [ ]:
client = clickhouse_connect.get_client(
    host='clickhouse',
    port=8123,  # HTTP interface
    username='abc',
    password='xyz'
)

In [3]:
client.command('DROP DATABASE IF EXISTS superficie_bd')

In [4]:
client.command("CREATE DATABASE IF NOT EXISTS superficie_bd") #creación de base de datos para bosques y cultivos 

In [5]:
print(client.query("SHOW DATABASES").result_rows)

[('INFORMATION_SCHEMA',), ('default',), ('information_schema',), ('superficie_bd',), ('system',)]


In [6]:
# Read CSV files
csv_path_0 = '/usr/workspace/media/00_superficie_forestal.csv'
data_0 = pd.read_csv(csv_path_0)

In [7]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 22 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   entidad_federativa                                        32 non-null     object 
 1   poblacion                                                 32 non-null     int64  
 2   superficie_total                                          32 non-null     float64
 3   superficie_forestal                                       32 non-null     float64
 4   superficie_no_forestal                                    32 non-null     float64
 5   superficie_con_arbolado                                   32 non-null     float64
 6   superficie_cubierta_por_bosque                            32 non-null     float64
 7   superficie_cubierta_por_selva                             32 non-null     float64
 8   superficie_cubierta_po

In [8]:
data_tuples_0 = [tuple(row) for row in data_0.values]
#data_tuples_0

In [9]:
# CREATE TABLE statement
create_table_0 = """
CREATE TABLE IF NOT EXISTS superficie_bd.superficie_forestal (
    entidad_federativa String,
    poblacion UInt32,
    superficie_total Float64,
    superficie_forestal Float64,
    superficie_no_forestal Float64,
    superficie_con_arbolado Float64,
    superficie_cubierta_por_bosque Float64,
    superficie_cubierta_por_selva Float64,
    superficie_cubierta_por_manglar Float64,
    superficie_cubierta_por_otras_areas_arboladas Float64,
    superficie_cubierta_por_matorral_xerofilo Float64,
    superficie_cubierta_por_otras_areas_forestales Float64,
    superficie_destinada_a_actividades_agricolas_de_humedad Float64,
    superficie_destinada_a_actividades_agricolas_de_riego Float64,
    superficie_destinada_a_actividades_agricolas_de_temporal Float64,
    superficie_cubierta_por_cuerpos_de_agua Float64,
    superficie_destinada_a_actividades_acuicolas Float64,
    superficie_de_pastizales_cultivados Float64,
    superficie_de_pastizales_inducidos Float64,
    superficie_sin_vegetacion_visible Float64,
    superficie_desprovista_de_vegetacion Float64,
    superficie_ocupada_por_asentamientos_humanos Float64
) ENGINE = MergeTree()
ORDER BY entidad_federativa
"""

In [10]:
# Execute the CREATE TABLE statement
try:
    client.command(create_table_0)
    print("Table 'superficie_forestal' created successfully!")
except Exception as e:
    print(f"Error creating table: {e}")

Table 'superficie_forestal' created successfully!


In [11]:
# Insert the DataFrame data
try:
    client.insert('superficie_bd.superficie_forestal', data_tuples_0)
    print(f"Successfully inserted {len(data_0)} rows into superficie_forestal table")
except Exception as e:
    print(f"Error inserting data: {e}")

Successfully inserted 32 rows into superficie_forestal table


In [12]:
result = client.query('''SELECT entidad_federativa, poblacion, round(superficie_cubierta_por_bosque/1000,1)
                        FROM superficie_bd.superficie_forestal
                        ORDER BY superficie_cubierta_por_bosque DESC''')

print(result.column_names)
result.result_rows

('entidad_federativa', 'poblacion', 'round(divide(superficie_cubierta_por_bosque, 1000), 1)')


[('Chihuahua', 3741869, 7007.3),
 ('Durango', 1832650, 5193.1),
 ('Oaxaca', 4132148, 3714.3),
 ('Jalisco', 8348151, 2366.0),
 ('Guerrero', 3540685, 2306.9),
 ('Sonora', 2944840, 1973.3),
 ('Michoacan', 4748846, 1652.5),
 ('Chiapas', 5543828, 1591.9),
 ('Zacatecas', 1622138, 1084.2),
 ('Nayarit', 1235456, 936.3),
 ('Sinaloa', 3026943, 870.6),
 ('Coahuila', 3146771, 836.6),
 ('Nuevo Leon', 5784442, 687.7),
 ('Estado de Mexico', 16992418, 623.6),
 ('Puebla', 6583278, 620.6),
 ('Tamaulipas', 3527735, 607.7),
 ('San Luis Potosi', 2822255, 569.4),
 ('Hidalgo', 3082841, 545.7),
 ('Guanajuato', 6166934, 477.2),
 ('Veracruz', 8062579, 316.0),
 ('Queretaro', 2368467, 280.1),
 ('Baja California Norte', 8769020, 176.5),
 ('Aguascalientes', 1425607, 101.6),
 ('Tlaxcala', 1342977, 66.7),
 ('Baja California Sur', 798447, 58.5),
 ('Morelos', 1971520, 54.2),
 ('Colima', 731391, 54.1),
 ('Ciudad de Mexico', 9209944, 41.0),
 ('Tabasco', 2402598, 19.7),
 ('Campeche', 928363, 13.5),
 ('Quintana Roo', 18579

In [15]:
result = client.query('''SELECT entidad_federativa, superficie_cubierta_por_selva
                            FROM superficie_bd.superficie_forestal
                            ORDER BY superficie_cubierta_por_selva DESC
                            LIMIT 1''')

print(result.column_names)
result.result_rows

('entidad_federativa', 'superficie_cubierta_por_selva')


[('Campeche', 3782854.848)]

In [16]:
result = client.query('''SELECT
                            entidad_federativa,
                            round(100*superficie_cubierta_por_bosque/poblacion,2) as indice_bosque_poblacion
                        FROM superficie_bd.superficie_forestal
                        ORDER BY
                            superficie_cubierta_por_bosque/poblacion DESC
                        ''')

print(result.column_names)
result.result_rows

('entidad_federativa', 'indice_bosque_poblacion')


[('Durango', 283.36),
 ('Chihuahua', 187.27),
 ('Oaxaca', 89.89),
 ('Nayarit', 75.79),
 ('Sonora', 67.01),
 ('Zacatecas', 66.84),
 ('Guerrero', 65.15),
 ('Michoacan', 34.8),
 ('Sinaloa', 28.76),
 ('Chiapas', 28.71),
 ('Jalisco', 28.34),
 ('Coahuila', 26.59),
 ('San Luis Potosi', 20.17),
 ('Hidalgo', 17.7),
 ('Tamaulipas', 17.23),
 ('Nuevo Leon', 11.89),
 ('Queretaro', 11.82),
 ('Puebla', 9.43),
 ('Guanajuato', 7.74),
 ('Colima', 7.4),
 ('Baja California Sur', 7.32),
 ('Aguascalientes', 7.13),
 ('Tlaxcala', 4.97),
 ('Veracruz', 3.92),
 ('Estado de Mexico', 3.67),
 ('Morelos', 2.75),
 ('Baja California Norte', 2.01),
 ('Campeche', 1.45),
 ('Tabasco', 0.82),
 ('Ciudad de Mexico', 0.44),
 ('Quintana Roo', 0.0),
 ('Yucatan', 0.0)]

In [17]:
result = client.query('''SELECT
                            entidad_federativa,
                            round(100*superficie_cubierta_por_matorral_xerofilo/superficie_total,2) as indice_matorral_xerofilo_superficie
                        FROM superficie_bd.superficie_forestal
                        ORDER BY
                            superficie_cubierta_por_matorral_xerofilo/superficie_total DESC
                        ''')

print(result.column_names)
result.result_rows

('entidad_federativa', 'indice_matorral_xerofilo_superficie')


[('Baja California Norte', 79.12),
 ('Baja California Sur', 75.83),
 ('Coahuila', 75.27),
 ('Sonora', 54.34),
 ('Nuevo Leon', 52.1),
 ('San Luis Potosi', 47.59),
 ('Zacatecas', 35.55),
 ('Chihuahua', 34.41),
 ('Tamaulipas', 26.49),
 ('Queretaro', 22.36),
 ('Durango', 21.26),
 ('Hidalgo', 13.43),
 ('Puebla', 8.18),
 ('Guanajuato', 6.89),
 ('Aguascalientes', 6.36),
 ('Sinaloa', 2.38),
 ('Colima', 1.67),
 ('Tlaxcala', 1.58),
 ('Estado de Mexico', 0.77),
 ('Ciudad de Mexico', 0.65),
 ('Oaxaca', 0.61),
 ('Jalisco', 0.19),
 ('Veracruz', 0.18),
 ('Nayarit', 0.16),
 ('Michoacan', 0.06),
 ('Morelos', 0.05),
 ('Chiapas', 0.0),
 ('Guerrero', 0.0),
 ('Quintana Roo', 0.0),
 ('Tabasco', 0.0),
 ('Campeche', 0.0),
 ('Yucatan', 0.0)]